In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy.integrate import solve_ivp
from scipy.interpolate import interp1d

import time
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
%matplotlib qt

In [ ]:
import sys
sys.path.append('..')
import neural_ode.NeuralODE

In [ ]:
import neural_ode.ODESolvers

In [ ]:
tf.keras.backend.set_floatx('float64')

In [ ]:
class SimpleModel(tf.keras.Model):
    def __init__(self, dyn_dim = 1, external_dim=1):
        super().__init__()
        w_init = tf.random_normal_initializer(mean=-1.0, stddev=0.05)
        self.w = tf.Variable(
            initial_value = w_init(shape=(dyn_dim*2, dyn_dim), dtype="float64"),
            trainable=True,
        )
        b_init = tf.zeros_initializer()
        self.b = tf.Variable(
            initial_value=b_init(shape=(dyn_dim,), dtype="float64"), trainable=True
        )
        self.dyn_dim = dyn_dim

    def call(self, inputs):
        x_ext = inputs[:,self.dyn_dim*2:]
        vels = inputs[:,self.dyn_dim:self.dyn_dim*2]
        accs = tf.matmul(inputs, self.w) + self.b
        return tf.concat([vels, accs], axis=1)


model = SimpleModel()

In [ ]:
n_ode = neural_ode.NeuralODE.NeuralODE(model, 2, 
                                       solver=neural_ode.ODESolvers.HeunsMethod())

In [ ]:
n_ode.model.variables

In [ ]:
n_ode.pretrain(t_scale=1.0, step_size=0.5, n_epoch = 1)

In [ ]:
# Generate solution

In [ ]:
#
N_n = int(2)
c = 0.1
k = 4.0
def oscilator(t, y):
    return np.array([y[1], -c*y[1]-k*y[0]])
t_final = 40.0
n_eval = int(1001)
t_span = np.array([0.0, t_final])
y0 = np.array([1.0, 0.0])

In [ ]:
sol = solve_ivp(oscilator, t_span, y0, t_eval=np.linspace(0, t_final, num=n_eval))

In [ ]:
plt.plot(sol.t, sol.y[0,:])

In [ ]:
# transform to tensorflow
t_span_tf = tf.constant(t_span)
y0_tf = tf.constant(y0, dtype=tf.float64)
t_target = tf.constant(sol.t)
y_target = tf.constant(np.transpose(sol.y) )

In [ ]:
# only displacements
y_target = tf.expand_dims(tf.constant(np.transpose(sol.y[0,:])), axis=1)

In [ ]:
sol1 = n_ode.forward_solve(t_target, y_target[0,:])
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [ ]:
y0 = tf.concat([y_target[0,0], (y_target[1,0]-y_target[0,0])/(t_target[1]-t_target[0])], axis=0)
sol1 = n_ode.forward_solve(t_target, y0)
fig = plt.figure()
ax = plt.gca()
ax.plot(t_target.numpy(), y_target[:,0].numpy())
ax.plot(sol1['t'].numpy(), sol1['y'][:,0].numpy())

In [ ]:
# fit

In [ ]:
n_epoch = 30
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_batch=10, 
          adjoint_method=False, missing_derivative=[0], adjust_initial=True)

In [ ]:
n_epoch = 30
n_ode.fit(t_target, y_target, n_epoch=n_epoch, n_batch=10, 
          adjoint_method=False, missing_derivative=[0], adjust_initial=False)

In [ ]:
n_ode.model.variables

In [ ]:
# Check derivatives

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.0, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss, dl_dp = n_ode.usual_method(t_target, y_target)
#loss, dL_dy, a = n_ode.adjoint_method(t_target, y_target)
#dl_dp = a[2:]

In [ ]:
dp = 0.00001
n_ode.model.variables[0].assign(np.array([[0.0, -k], [1.1, -c+dp]]))
n_ode.model.variables[1].assign(np.array([0,0]))

In [ ]:
loss2, dl_dp2 = n_ode.usual_method(t_target, y_target)

In [ ]:
(loss2-loss)/dp

In [ ]:
dl_dp